## Import statements

In [ ]:
from datasets import load_dataset
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from transformers import AutoTokenizer, PreTrainedTokenizerFast

## Data loading

In [ ]:
dataset = load_dataset("", split="train")


def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i: i + 1000]["text"]

## Initialization/training

In [ ]:
tokenizer = Tokenizer(models.BPE(unk_token="<UNK>"))
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
special_tokens = ["<PAD>", "<EOS>", "<UNK>"]
trainer = trainers.BpeTrainer(
    vocab_size=300,
    special_tokens=special_tokens,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

## Testing

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)
print(encoding.ids)
print(tokenizer.decode(encoding.ids))
print(len(tokenizer.get_vocab()))
print(tokenizer.get_vocab_size())

## Saving

In [ ]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    pad_token="<PAD>",
    eos_token="<EOS>",
    unk_token="<UNK>",
    # cls_token="[CLS]",
    # sep_token="[SEP]",
    # mask_token="[MASK]",
)

wrapped_tokenizer.save_pretrained("../../saved_models/tokenizers/rotten_tomatoes_bpe_style")

In [ ]:
tok = AutoTokenizer.from_pretrained("../../../saved_models/tokenizers/rotten_tomatoes/rotten_tomatoes_bpe_style")

tokens = tok.tokenize("Test number 2.!@#$%^&*()")

print(tokens)
ids = tok.convert_tokens_to_ids(tokens)
print(ids)
decoded_string = tok.decode(ids)
print(decoded_string)
print(len(tok.get_vocab()))